### Mask Inclusion Raster
In this notebook, we mask our 'Technical Inclusion' layer with (1) the 'Environmental Exclusion' layer and (2) the 'Techhnical Exclusion' layer to generate our InclusionArea_Solar_SL1_ZRM.tif and InclusionArea_Wind_SL1_ZRM.tif rasters. We then extract the output using Tiger/Line 2018 CONUS boundaries, and convert the raster datasets to vector feature classes, and dissolve the feature classes. 

**Note:** The current version requires the user to manually review and delete coastal waters polygons before running the dissolve_feature_class() function (for most applications). This will be corrected in future versions.  

In [1]:
import arcpy
import os
from arcpy import env
from arcpy.sa import *

#### Define useful functions

In [ ]:
def raster_calculation(first_raster, second_raster, third_raster, output_raster):
    """    
    This function creates a raster object from the provided input paths and constructs a conditional expression. 
    """
    
    try:
        # Create a raster object from the input path
        raster1 = arcpy.sa.Raster(first_raster)
        raster2 = arcpy.sa.Raster(second_raster)
        raster3 = arcpy.sa.Raster(third_raster)

        # Build the arcpy.sa.Con expression
        expression = arcpy.sa.Con(((raster1 + arcpy.sa.Con(arcpy.sa.IsNull(raster2), 0, raster2) + arcpy.sa.Con(arcpy.sa.IsNull(raster3), 0, raster3)) == 1), 1, arcpy.sa.SetNull(raster1, 0))
        expression.save(output_raster)

        print(f"Raster calculation completed successfully. Result saved to {output_raster}")

    except arcpy.ExecuteError:
        print(arcpy.GetMessages())
    except Exception as e:
        print(f"An error occurred: {str(e)}")

In [ ]:
def raster_to_feature_class(input_raster, output_feature_class, dissolve_field="VALUE"):
    """
    Convert a raster to a polygon feature class and provide usage instructions for further processing.
    """
    
    try:
        # Convert the raster to a polygon feature class
        arcpy.RasterToPolygon_conversion(input_raster, output_feature_class, "NO_SIMPLIFY", "VALUE")

        print(f"Raster converted to feature class successfully. Please use your GIS to remove coastal waters polygons by hand before dissolving.")

    except arcpy.ExecuteError:
        print(arcpy.GetMessages())
    except Exception as e:
        print(f"An error occurred: {str(e)}")

In [ ]:
def remove_small_polygons(input_feature_class, output_feature_class, area_threshold_sqm=2000000):
    """
    Remove small polygons from a feature class based on a defined area requirement.
    """

    try:
        # Use a definition query to select polygons with an area greater than or equal to the threshold
        definition_query = f"SHAPE_Area >= {area_threshold_sqm}"
        arcpy.MakeFeatureLayer_management(input_feature_class, "lyr_selected", definition_query)

        # Copy the selected polygons to the output feature class
        arcpy.CopyFeatures_management("lyr_selected", output_feature_class)

        print(f"Small polygons with an area less than {area_threshold_sqm} sqm have been removed.")

    except arcpy.ExecuteError:
        print(arcpy.GetMessages())
    except Exception as e:
        print(f"An error occurred: {str(e)}")

In [ ]:
def dissolve_feature_class(input_feature_class, output_feature_class):
    """
    Dissolve features in a feature class, repairing geometry if necessary and recalculating the extent.
    """
    
    try:
        # Attempt to dissolve the features without specifying a dissolve field
        arcpy.Dissolve_management(input_feature_class, output_feature_class)

        print("Features dissolved successfully.")

    except arcpy.ExecuteError as e:
        print(f"Failed to dissolve features: {e}")

        # Repair geometry to fix any invalid geometries
        arcpy.RepairGeometry_management(input_feature_class)

        print("Geometry repaired successfully.")

        # Recalculate the feature class extent to the minimum bounding rectangle of the features
        arcpy.CalculateGeometryAttributes_management(input_feature_class, [["EXTENT", "POLYGON"]])

        print("Feature class extent recalculated successfully.")

        try:
            # Attempt to dissolve again after repairing the geometry
            arcpy.Dissolve_management(input_feature_class, output_feature_class)

            print("Features dissolved successfully after geometry repair.")

        except arcpy.ExecuteError as e:
            print(f"Failed to dissolve features after geometry repair: {e}")

#### Set input and output paths, set environmental variables for raster analysis

In [2]:
# Set path to the main input folder
mainInputFolder = "C:\\Users\\Zachary\\ASSET\\supplyCurve\\analysis\\data"

In [3]:
# Set path to environmental exclusion rasters
environmental_exclusion_raster = os.path.join(mainInputFolder, "environmentalExclusionRasters\\Mosaic\\environmentalExclusionWind_SL1_ZRM.tif")

In [4]:
# Set path to technical threshold rasters
technical_threshold_raster = os.path.join(mainInputFolder, "technicalExclusionRasters\\Mosaic\\technicalExclusionMosaic_Wind.tif")

In [5]:
# Set path to technical inclusion area raster
technical_inclusion_area = os.path.join(mainInputFolder, "technicalInclusionAreas\\technicalInclusionArea_Wind_binary.tif")

In [6]:
# Set path for raster calculator output
inclusion_area = os.path.join(mainInputFolder, "myResults\\modelOutputs\\InclusionArea_Wind_SL1_ZRM.tif")

In [7]:
# Set path for inclusion area feature class
inclusion_area_fc = os.path.join(mainInputFolder, "inclusionAreas.gdb\\InclusionArea_Wind_SL1_ZRM")

In [8]:
# Set path for inclusion area (with size constraint) feature class
inclusion_area_fc_dq = inclusion_area_fc + "_size_constraint"
inclusion_area_fc_dq

'C:\\Users\\Zachary\\ASSET\\supplyCurve\\analysis\\data\\inclusionAreas.gdb\\InclusionArea_Wind_SL1_ZRM_size_constraint'

In [ ]:
# Set path for dissolved feature class
inclusion_area_fc_dq_dissolved = inclusion_area_fc_dq + "_dissolved"

In [74]:
# set environmental raster settings to the reclassified slope raster
arcpy.env.snapRaster = technical_threshold_raster
arcpy.env.extent = technical_threshold_raster
arcpy.env.cellSize = technical_threshold_raster
arcpy.env.mask = os.path.join(mainInputFolder, "other\\cb_2018_us_state_500k_project.shp")

In [ ]:
raster_calculation(technical_inclusion_area, environmental_exclusion_raster, technical_threshold_raster, inclusion_area)

In [ ]:
raster_to_feature_class(inclusion_area, inclusion_area_fc)

In [ ]:
remove_small_polygons(inclusion_area_fc, inclusion_area_fc_dq)

In [ ]:
dissolve_feature_class(inclusion_area_fc_dq, inclusion_area_fc_dq_dissolved)